This notebook builds a function such that:
- input: aritst
- output: their genres

Run all the cells leading up to the function and then you can put a genre into the function and run that cell.


This function will be turned into a web app using streamlit for publisc exploration of the dataset

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
%matplotlib inline
#%matplotlib notebook

import re

from functools import partial

Import the cleaned data:

In [2]:
%ls -lt ../../data/genre_lists/data_ready_for_model/

total 39824
-rw-r--r--  1 Daniel  staff    21724 Jun  9 11:47 genre_label_counts_TOTAL_2020-05-18-10-06.csv
-rw-r--r--@ 1 Daniel  staff   287510 Jun  4 13:42 genre_stats.html
-rw-r--r--@ 1 Daniel  staff   911587 Jun  4 13:39 genre_set_counts.html
-rw-r--r--@ 1 Daniel  staff     1845 Jun  4 13:11 female_bias_freq500.html
-rw-r--r--@ 1 Daniel  staff     1459 Jun  4 13:11 male_bias_freq500.html
-rw-r--r--  1 Daniel  staff    73746 May 29 10:19 genre_stats.csv
-rw-r--r--  1 Daniel  staff    66235 May 21 11:00 promiscuity_table.csv
-rw-r--r--  1 Daniel  staff    57474 May 20 12:47 corpus.mm.index
-rw-r--r--  1 Daniel  staff   382436 May 20 12:47 corpus.mm
-rw-r--r--  1 Daniel  staff    49966 May 20 12:47 genre_dictionary.dict
drwxr-xr-x  5 Daniel  staff      160 May 20 10:59 logistic_model_data/
-rw-r--r--  1 Daniel  staff    10926 May 18 11:10 genre_label_non-lonely_TRAINING_2020-05-18-10-06.csv
-rw-r--r--  1 Daniel  staff     8664 May 18 11:09 genre_label_lonely_TRAINING_2020-05-18-10-06.

In [3]:
%store -r now
now
#now = '2020-05-11-14-35'

'2020-05-18-10-06'

In [4]:
X_train = pd.read_csv('/Users/Daniel/Code/Genre/data/genre_lists/data_ready_for_model/wiki-kaggle_X_train_{}.csv'.format(now), index_col = ['artist'])
y_train = pd.read_csv('/Users/Daniel/Code/Genre/data/genre_lists/data_ready_for_model/wiki-kaggle_y_train_{}.csv'.format(now), index_col = ['artist'])
X_test = pd.read_csv('/Users/Daniel/Code/Genre/data/genre_lists/data_ready_for_model/wiki-kaggle_X_test_{}.csv'.format(now), index_col = ['artist'])
y_test = pd.read_csv('/Users/Daniel/Code/Genre/data/genre_lists/data_ready_for_model/wiki-kaggle_y_test_{}.csv'.format(now), index_col = ['artist'])

In [5]:
X_tot = pd.concat([X_train,X_test])
y_tot = pd.concat([y_train,y_test])

In [6]:
X_tot.shape, y_tot.shape

((15470, 2), (15470, 1))

In [7]:
data = y_tot.join([X_tot], how = 'outer')

In [8]:
data.head()

,gender,genrelist,genrelist_length
artist,,,
Pablo_Holman,male,"['pop', 'rock', 'emo_pop']",3
Bobby_Edwards,male,['country'],1
La_Palabra,male,"['afro_cuban_jazz', 'son_montuno', 'guaracha',...",4
Sherrick,male,"['r_and_b', 'soul']",2
Allen_Collins,male,['southern_rock'],1


In [9]:
data.shape, data.isnull().sum()

((15470, 3),
 gender              0
 genrelist           0
 genrelist_length    0
 dtype: int64)

### Genre Labels

Each value of the genre column is a _string_ of comma separated genre labels. We want to convert it to a _list_ of strings.

In [10]:
"""This function takes in a string of the form
appearing in the genrelist of the dataframe.
It strips the square brackets and extra quotes and
returns a list of strings where each string is a genre label."""
def genrelist(string):
    string = string.strip("[").strip("]").replace("'","")
    L = [s for s in string.split(',')]
    L_new = []
    for x in L:
        L_new.append(x.replace(" ","_").lstrip("_").rstrip("_"))
    while (str("") in L_new):
        L_new.remove("")
    return L_new

Now we apply it to the whole column and put the lists in a new column:

In [11]:
data['genrelist']= data['genrelist'].apply(genrelist)

In [12]:
data.head()

,gender,genrelist,genrelist_length
artist,,,
Pablo_Holman,male,"[pop, rock, emo_pop]",3
Bobby_Edwards,male,[country],1
La_Palabra,male,"[afro_cuban_jazz, son_montuno, guaracha, salsa...",4
Sherrick,male,"[r_and_b, soul]",2
Allen_Collins,male,[southern_rock],1


## Co-Occurrence

Function to apply to dataframe that marks a row as having a genre label:

In [48]:
def genres_of_an_artist(data, artist_name = 'La_Palabra'):
    genres = data.loc[artist_name, 'genrelist']
    genres = ", ".join(map(str,genres))
    return genres.title()

In [49]:
genres_of_an_artist(data, 'Sherrick')

'R_And_B, Soul'

In [24]:
data.sort_index().index.values.tolist()

['12_Gauge',
 '21_Savage',
 '2_Chainz',
 '2_Pistols',
 '40_Cal.',
 '40_Glocc',
 '50_Cent',
 '6LACK',
 '88-Keys',
 '9th_Prince',
 '9th_Wonder',
 'A+',
 'A-Plus',
 'A.B._Quintanilla',
 'A.C._Marias',
 'A.D.O.R.',
 'A._A._Bondy',
 'A._C._Newman',
 'A._J._Croce',
 'A._J._Ghent',
 'A._J._Masters',
 'A._L._Lloyd',
 'A._P._Carter',
 'AJ_McLean',
 'AJ_Michalka',
 'AM',
 'AMG',
 'AR-Ab',
 'ASAP_Ferg',
 'ASAP_Nast',
 'ASAP_Rocky',
 'ASAP_Twelvyy',
 'ASAP_TyY',
 'AZ',
 'A_Boogie_wit_da_Hoodie',
 'Aaliyah',
 'Aaradhna',
 'Aaron_Barker',
 'Aaron_Barrett',
 'Aaron_Bedard',
 'Aaron_Benward',
 'Aaron_Bertram',
 'Aaron_Bruno',
 'Aaron_Carter',
 'Aaron_Deer',
 'Aaron_Gillespie',
 'Aaron_Hall',
 'Aaron_Kelly',
 'Aaron_Lee_Tasjan',
 'Aaron_Lewis',
 'Aaron_Lines',
 'Aaron_North',
 'Aaron_Pauley',
 'Aaron_Pritchett',
 'Aaron_Riches',
 'Aaron_Stainthorpe',
 'Aaron_Tippin',
 'Aaron_Turner',
 'Aaron_Walpole',
 'Aaron_Watson',
 'Ab-Soul',
 'Abbey_Lincoln',
 'Abel_Pintos',
 'Abel_Talamántez',
 'Abeti_Masikini',


In [45]:
mylist = ['aih','b','b']
", ".join(mylist)

'aih, b, b'